In [249]:
# Productionalize Data Scraping WHL.CA
# 1. Convert ipynb files to py files
# 2. Copy py files to server
# 3. Chmod server files

In [250]:
import subprocess
import sys
sys.path.append('../../')

import settings as st

import paramiko

In [251]:
"""
script_list = [
    'season_list',
    'schedule_by_season',
    'team_list_by_season',
    'team_roster_by_season',
    'player_profile',
    'player_season_stats',
    'player_game_stats',
    'game_summary_officials',
    'game_summary_shootout',
    'game_summary_penalties',
    'game_summary_team',
    'game_summary_meta',
    'game_summary_periods',
    'game_summary_goalies',
    'game_summary_lineup',
    'game_summary_coaches',
    'game_summary_goals'
]
"""

script_list = [
    'schedule_team_by_season'
]

In [252]:
"""
Convert the specified list of notebooks to their corresponding targets
"""

def convert_notebook_to_py(source, target):
    subprocess.run(["jupyter", "nbconvert", 
                "--to", "script", source, 
                "--output", target])
    
for p in script_list:
    source = '/Users/jedidiahong/Documents/GitHub/hitmen-hockey/notebooks/data-scraping/whl.ca/{0}.ipynb'.format(p)
    target = '/Users/jedidiahong/Documents/GitHub/hitmen-hockey/production/data-scraping/whl.ca/{0}'.format(p)
    
    convert_notebook_to_py(source, target)

In [253]:
sys.path.append('../../production/data-scraping/utilities')

import encoder_utilities as eu

prod_host = st.vm['do-ubuntu']['host']
prod_user = st.vm['do-ubuntu']['user']
prod_pass = eu.decode(st.vm['do-ubuntu']['pass'])

In [254]:
"""
Update Scripts to production paths and imports
"""

for p in script_list:
    source = '/Users/jedidiahong/Documents/GitHub/hitmen-hockey/production/data-scraping/whl.ca/{0}.py'.format(p)
    target = '/Users/jedidiahong/Documents/GitHub/hitmen-hockey/production/data-scraping/whl.ca/tmp/{0}.py'.format(p)

    with open(source) as f:
        lines = f.readlines()
            
    target_file = open(target,"w+")
    
    for line in lines:
        
        if line.strip() == "sys.path.append('../../../')":
            line = "sys.path.append('/home/do-user/projects/hitmen_hockey/')\n"
        
        if line.strip() == "sys.path.append('../../../production/data-scraping/utilities')":
            line = "sys.path.append('/home/do-user/projects/hitmen_hockey/production/data-scraping/utilities')\n"
        
        if line.strip().startswith("pd.set_option"):
            line = ""
            
        if line.strip().startswith("get_ipython"):
            line = ""
            
        target_file.write(line)
        
    target_file.close()
    
for p in script_list:
    source = '/Users/jedidiahong/Documents/GitHub/hitmen-hockey/production/data-scraping/whl.ca/tmp/{0}.py'.format(p)
    target = '/Users/jedidiahong/Documents/GitHub/hitmen-hockey/production/data-scraping/whl.ca/{0}.py'.format(p)
    
    subprocess.run(["mv", source, target])

In [255]:
"""
Copy the specified list of scripts to the server
"""

import paramiko
from scp import SCPClient

def createSSHClient(server, port, user, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(server, port, user, password)
    return client

print(prod_host)

ssh = createSSHClient(prod_host, 22, prod_user, prod_pass)
scp = SCPClient(ssh.get_transport())

for p in script_list:
    print(p)
    
    source = '/Users/jedidiahong/Documents/GitHub/hitmen-hockey/production/data-scraping/whl.ca/{0}.py'.format(p)
    target = '/home/do-user/projects/hitmen_hockey/production/data-scraping/whl.ca/{0}.py'.format(p)
    
    scp.put(source, target)
    
scp.close()

167.99.182.232
schedule_team_by_season


In [256]:
"""
Chmod for the list of scripts copied to the server
"""

def executeSSHCommand(server, port, user, password, command):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    client.connect(server, port, user, password)
    shell = client.invoke_shell()
    
    _, stdout, stderr = client.exec_command(command)
    
    stdout = stdout.read().decode()
    stderr = stderr.read().decode()

    shell.close()
    client.close()

    return stdout, stderr

for p in script_list:
    file = '/home/do-user/projects/hitmen_hockey/production/data-scraping/whl.ca/{0}.py'.format(p)
    command = "chmod 755 {0}".format(file)
    
    stdout = executeSSHCommand(prod_host, 22, prod_user, prod_pass, command)
    print(stdout)

('', '')
